# Introduction:


Problem: There are many people who have restrictions on what they can eat, the reasons for which are numerous. For example some people may be vegatarian, vegan, have religious restrictions, allergic to certain foods, etc and may need a specialty restaurant that can provide meals that satisfy their food requirements. This issue most prominently occurs when people visit a far away city, state or even a country whose food culture is vastly different from the one that they are use to. The problem that I am looking to solve is finding areas within a city that can best accomodate a person's food restrictions/diets/preferences.

Audience: One of the many variables that a person goes through when they are searching for a new home or apartment to live in is the food options available to them whether that is the nearby restaurants they can dine out at or grocery stores that carry the ingrediants that they are looking for. And as mentioned earlier, this information can also be used by people who are visiting a city on vacation and may be worried that their food options will be limited or possibly non-existent. They can then use information like this to prepare and find other areas with restaurants or stores that can fulfill their food needs.

For this project I will focus on the city of San Francisco

In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes

In [69]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import requests
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import folium
from folium.plugins import HeatMap

In [62]:
#Foursquare
CLIENT_ID = 'KNCYXV5LQSPDYOZEKBGQPYDL130HCNE3S5MAHVAGQQP5XDZQ' # your Foursquare ID
CLIENT_SECRET = '5P3NUVYMEPEHBOILBZHLRGAFK13FBW4SY3LQBH5FNRSW1WEX' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#Yelp
Y_CLIENT_ID = 'boXqe5m1YRipY72wL4RmHw'
API_KEY = 'uTD08rDrA5eiFTEoXR8TmcjXZiZQu0vzUmCGfZssh25BWpgVrSwkc-EOHOV8rt_dIAU5s7xdadp9aC5DozqSjz85SkCPX1V-CZcgzH1Fir48KnSVmx4P16CRg8agXXYx'

Your credentails:
CLIENT_ID: KNCYXV5LQSPDYOZEKBGQPYDL130HCNE3S5MAHVAGQQP5XDZQ
CLIENT_SECRET:5P3NUVYMEPEHBOILBZHLRGAFK13FBW4SY3LQBH5FNRSW1WEX


# Data:

#### The data that I need to solve the problem is the Name, location, and the category of the restaurant.

#### By default the Nominatim Geolocator will pinpoint to Downtown San Francisco, but for my project I will need to get different lat,lng coordinates to be able to search different neighbourhoods and areas around San Francisco. Foursquare API has a 50 return limit on the search query. So in order to get the data I need I will have to tune the search query by changing the lat, lng coordinates, the radius of search, as well as the foursquare ID of the restaurant category(i.e. vegetarian) to get a better idea of the type of food coverage around the city

#### The different areas that were selected were based off the Supervisorial Districts as officially designated by the city of San Francisco

In [5]:
#list of Lat, Lng Coordinates for the different areas of San Francisco
#Financial District / Soma / Civic Center: 37.777793, -122.409237 ; 37.788444, -122.396117
#Sunset(Outer/Inner): 37.759023, -122.495649 ; 37.740224, -122.494320 ; 37.760308, -122.466106
#Mission District / Bernal Heights: 37.761931, -122.417072 ; 37.753921, -122.416531 ; 37.743804, -122.418943
#Richmond(Outer/Inner): 37.777956, -122.489961 ; 37.781263, -122.479482 ; 37.782970, -122.464397
#Presidio/Marina/Pacific Heights: 37.799706, -122.435992 ; 37.788965, -122.433864
#Fisherman's Wharf / Chinatown / Haight: 37.803146, -122.411292 ; 37.794611, -122.408020 ; 37.771439, -122.440401
#Lakeshore / Ingleside / Ocean View: 37.726858, -122.472065 ; 37.717707, -122.449355
#Twin Peaks / Castro / Noe Valley: 37.765102, -122.435335 ; 37.751354, -122.434074 ; 37.745580, -122.451508
#Potrero Hill / Bayview: 37.762299, -122.401388; 37.760476, -122.388917 ; 37.734800, -122.390652 ; 37.720093, -122.407951


coordinates = [(37.788444, -122.396117),
               (37.777793, -122.409237),
              (37.759023, -122.495649),
              (37.740224, -122.494320),
              (37.760308, -122.466106),
              (37.761931, -122.417072),
              (37.753921, -122.416531),
              (37.743804, -122.418943),
              (37.777956, -122.489961),
              (37.781263, -122.479482),
              (37.782970, -122.464397),
              (37.799706, -122.435992),
               (37.788965, -122.433864),
              (37.803146, -122.411292),
             (37.794611, -122.408020),
               (37.771439, -122.440401),
              (37.726858, -122.472065),
               (37.717707, -122.449355),
              (37.765102, -122.435335),
              (37.751354, -122.434074),
               (37.745580, -122.451508),
               (37.762299, -122.401388),
               (37.760476, -122.388917),
              (37.734800, -122.390652),
              (37.720093, -122.407951)]

radius = 3000
LIMIT = 100

#### Create a list of foursquare URL search queries for all the different lat, lng coordinates. Once for vegetarian restaurants and once for Halal Restaurants

In [288]:
v_all_url = []
h_all_url = []
vegetarian_vegan_id = '4bf58dd8d48988d1d3941735'
halal_id = '52e81612bcbc57f1066b79ff'

for latitude, longitude in coordinates:
    v_all_url.append('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude, 
    VERSION,
    vegetarian_vegan_id,
    radius, 
    LIMIT))
    
    h_all_url.append('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude, 
    VERSION,
    halal_id,
    radius, 
    LIMIT))

#### Concatenate all the data we got and form one combined Dataframe

In [7]:
df = []
for url in v_all_url:
    results = requests.get(url).json()
    venues = results['response']['venues']
    dataframe = json_normalize(venues)
    df.append(dataframe)
    
combined_df = pd.concat(df, axis = 0, sort = True)

In [8]:
hdf = []
for url in h_all_url:
    results = requests.get(url).json()
    venues = results['response']['venues']
    dataframe = json_normalize(venues)
    hdf.append(dataframe)
    
h_combined_df = pd.concat(hdf, axis = 0, sort = True)

#### Clean the data to make it more presentable and easier to use

In [ ]:
#Vegeatarian Restaurants Data Cleaning
processed_df = combined_df[['categories', 'name', 'location.address','location.lat', 'location.lng']]
processed_df.drop_duplicates(subset = 'location.lat', keep = 'first', inplace = True)
processed_df.reset_index(drop=True, inplace = True)
processed_df.rename(columns = {'name' : 'Name', 
                               'location.address' : 'Address', 
                               'location.lat' : 'Latitude', 
                               'location.lng' : 'Longitude'
                              }, inplace = True)

#Halal Restaurants Data Cleaning
h_processed_df = h_combined_df[['categories', 'name', 'location.address','location.lat', 'location.lng']]
h_processed_df.drop_duplicates(subset = 'location.lat', keep = 'first', inplace = True)
h_processed_df.reset_index(drop=True, inplace = True)
h_processed_df.rename(columns = {'name' : 'Name', 
                               'location.address' : 'Address', 
                               'location.lat' : 'Latitude', 
                               'location.lng' : 'Longitude'
                              }, inplace = True)

In [27]:
#Extract Category of the restaurant from dataframe
lod = processed_df['categories']
hlod = h_processed_df['categories']
category = []
h_category = []
for lista in lod:
    #This If statement checks for any row entries that have an empty value and if so adds a NaN to the categories list
    if not lista:
        category.append(np.nan)
    for dictionary in lista:
        category.append(dictionary['name'])
        
processed_df.insert(loc = 2, column = "Category", value = category)

for lista in hlod:
    if not lista:
        h_category.append(np.nan)
    for dictionary in lista:
        h_category.append(dictionary['name'])
        
h_processed_df.insert(loc = 2, column = "Category", value = h_category)

In [28]:
processed_df = processed_df.drop(columns = ['categories'])
h_processed_df = h_processed_df.drop(columns = ['categories'])

#### The results for Vegetarian Restaurants returned 71 restaurants and 14 for Halal Restaurants

In [317]:
processed_df

,Name,Category,Address,Latitude,Longitude
0,Mixt,Salad Place,560 Mission St,37.788844,-122.399149
1,sweetgreen,Salad Place,171 2nd St,37.787262,-122.398635
2,Wildseed,Vegetarian / Vegan Restaurant,2000 Union St,37.797676,-122.432420
3,"Stirred, Not Shaken",Bakery,NaN,37.761962,-122.417972
4,The Plant Cafe Organic,Café,101 California St,37.792525,-122.398465
5,Mixt,Salad Place,901 Valencia St,37.758400,-122.421126
6,Gracias Madre,Mexican Restaurant,2211 Mission St,37.761484,-122.419144
7,Shizen,Sushi Restaurant,370 14th St,37.768190,-122.421627
8,Vegan Picnic Polk St,Vegetarian / Vegan Restaurant,1323 Polk St,37.788959,-122.420448
9,Elixiria,Vegetarian / Vegan Restaurant,25 Beale St,37.792053,-122.396844


In [318]:
h_processed_df

,Name,Category,Address,Latitude,Longitude
0,Torshi,Halal Restaurant,"Leidesdorff Alley, 343 Sansome St",37.793609,-122.402095
1,golden gate halal food,Halal Restaurant,979 Market St,37.782475,-122.409757
2,The Halal Guys,Halal Restaurant,340 O'Farrell St,37.786111,-122.410421
3,Marhaba Indian & Pakistani Halal Cuisine,Halal Restaurant,485 Ofarrell St,37.785743,-122.412633
4,Eden Silk Road,Chinese Restaurant,575 Ofarrell St,37.785731,-122.414099
5,The Halal Guys,Halal Restaurant,1022 Bush St,37.789591,-122.414238
6,Hella Halal,Halal Restaurant,320 11th,37.771313,-122.413594
7,SF Gyro,Food Stand,NaN,37.777250,-122.416502
8,Curry Leaf Restaurant,Indian Restaurant,943 Columbus Ave,37.803183,-122.414413
9,Maykadeh Persian Cuisine,Persian Restaurant,470 Green St,37.799788,-122.407118


# Methodology:

#### Here I will use Folium to generate a map of San Francisco and plot the list of restaurants that we retrieved earlier from FourSquare. This first map will be for the Vegetarian/Vegan restaurants

In [331]:
map_sf = folium.Map(location=[37.788444, -122.396117], zoom_start = 12)

for lat, lng, name, category, address in zip(processed_df['Latitude'], 
                                             processed_df['Longitude'], 
                                             processed_df['Name'], 
                                             processed_df['Category'], 
                                             processed_df['Address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sf)
    
restaurant_coords = tuple(zip(processed_df['Latitude'], processed_df['Longitude']))

HeatMap(restaurant_coords).add_to(map_sf)
map_sf

#### Next the map for Halal Restaurants

In [324]:
h_map_sf = folium.Map(location=[37.788444, -122.396117], zoom_start = 12)

for lat, lng, name, category, address in zip(h_processed_df['Latitude'], 
                                             h_processed_df['Longitude'], 
                                             h_processed_df['Name'], 
                                             h_processed_df['Category'], 
                                             h_processed_df['Address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(h_map_sf)
    
#h_restaurant_coords = tuple(zip(h_processed_df['Latitude'], h_processed_df['Longitude']))

#HeatMap(h_restaurant_coords).add_to(h_map_sf)
h_map_sf

#### Upon looking at the map I thought that it looked like there was very little compared to the amount of Halal Restaurants I thought there would be in San Francisco. Because of that I wanted to double check with different sources to see if either I or FourSquare missed a lot of restaurants

#### I cross checked my FourSquare list with Google Maps Search as well as Yelp and found that a lot of restaurants were not listed in the FourSquare search return that I found on the other two. I looked into why and it seems that Foursquare only returned restaurants explicitly listed as a Vegan / Vegetarian Restaurants whereas Yelp would return any restaurant that offers a Vegetarian / Vegan dish

In [302]:
y_all= []
y_h_all = []
for latitude, longitude in coordinates:
    headers = {'Authorization': 'Bearer %s' % API_KEY}
    y_url='https://api.yelp.com/v3/businesses/search'
    params = {'term':'Vegetarian','latitude' : latitude, 'longitude' : longitude, 'limit' : '50'}
    y_results = requests.get(y_url, params=params, headers=headers).json()
    y_all.append(y_results)
    
    y_h_url='https://api.yelp.com/v3/businesses/search'
    params = {'term':'Halal','latitude' : latitude, 'longitude' : longitude, 'limit' : '50'}
    y_h_results = requests.get(y_h_url, params=params, headers=headers).json()
    y_h_all.append(y_h_results)


In [309]:
y_collection = []
y_category = []

y_h_collection = []
y_h_category = []

for result in y_all:
    y_venues = result['businesses']
    y_dataframe = json_normalize(y_venues)

    y_lod = y_dataframe['categories']
    for lista in y_lod:
        gather = []
        if not lista:
            y_category.append(np.nan)
        for dictionary in lista:
            gather.append(dictionary['title'])
        y_category.append(gather)
    y_collection.append(y_dataframe)
    
for result in y_h_all:
    y_venues = result['businesses']
    y_h_dataframe = json_normalize(y_venues)

    y_lod = y_h_dataframe['categories']
    for lista in y_lod:
        gather = []
        if not lista:
            y_h_category.append(np.nan)
        for dictionary in lista:
            gather.append(dictionary['title'])
        y_h_category.append(gather)
    y_h_collection.append(y_h_dataframe)

y_combined_df = pd.concat(y_collection, axis = 0, sort = True)
y_combined_df = y_combined_df[['name',  'location.address1', 'coordinates.latitude', 'coordinates.longitude']]
y_combined_df.insert(loc = 1, column = "Category", value = y_category)
y_combined_df['Category'] = [i[0] for i in y_combined_df['Category']]

y_h_combined_df = pd.concat(y_h_collection, axis = 0, sort = True)
y_h_combined_df = y_h_combined_df[['name',  'location.address1', 'coordinates.latitude', 'coordinates.longitude']]
y_h_combined_df.insert(loc = 1, column = "Category", value = y_h_category)
y_h_combined_df['Category'] = [i[0] for i in y_h_combined_df['Category']]

In [311]:
y_combined_df.drop_duplicates(subset = 'coordinates.latitude', keep = 'first', inplace = True)
y_combined_df.reset_index(drop=True, inplace = True)
y_combined_df.rename(columns = {'name' : 'Name',
                               'location.address1' : 'Address',
                               'coordinates.latitude' : 'Latitude',
                               'coordinates.longitude' : 'Longitude'}, inplace = True)

y_h_combined_df.drop_duplicates(subset = 'coordinates.latitude', keep = 'first', inplace = True)
y_h_combined_df.reset_index(drop=True, inplace = True)
y_h_combined_df.rename(columns = {'name' : 'Name',
                               'location.address1' : 'Address',
                               'coordinates.latitude' : 'Latitude',
                               'coordinates.longitude' : 'Longitude'}, inplace = True)

#### Update the Vegetarian / Vegan Map to include the Restaurants found on Yelp marked as Red Circles

In [326]:
for lat, lng, name, category, address in zip(y_combined_df['Latitude'], 
                                             y_combined_df['Longitude'], 
                                             y_combined_df['Name'], 
                                             y_combined_df['Category'], 
                                             y_combined_df['Address']):
    label = '{}, {}, {}'.format(name, category, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sf)
    
map_sf

In [327]:
for lat, lng, name, category, address in zip(y_h_combined_df['Latitude'], 
                                             y_h_combined_df['Longitude'], 
                                             y_h_combined_df['Name'], 
                                             y_h_combined_df['Category'], 
                                             y_h_combined_df['Address']):
    label = '{}, {}, {}'.format(name, category, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(h_map_sf)

#### Combine the FourSquare and Yelp data together removing any duplicates by using their Address. Removing duplicates by name would result in chain restaurants being removed so I could not use that. I also found that the latitude and longitude values between FourSquare and Yelp differed very slightly which would result in no duplicates being found if dropping duplicates through coordinates

In [280]:
df_list = [y_combined_df, processed_df]
concat_lists = pd.concat(df_list, axis = 0, sort = True)
concat_lists.drop_duplicates(subset = 'Address', keep = 'first', inplace = True)
concat_lists.reset_index(drop=True, inplace = True)

In [289]:
concat_lists

,Address,Category,Latitude,Longitude,Name
0,854 Washington St,Vegetarian,37.795106,-122.407379,Lucky Creation Vegetarian Restaurant
1,2000 Union St,Vegetarian,37.797687,-122.432420,Wildseed
2,2 Marina Blvd,Vegetarian,37.805322,-122.432264,Greens Restaurant
3,1298 Market St,Vegetarian,37.777980,-122.416200,Ananda Fuara
4,149 2nd St,Laotian,37.787514,-122.398720,Lao Table
5,839 Kearny St,Chinese,37.795906,-122.405247,Enjoy Vegetarian Restaurant
6,370 14th St,Sushi Bars,37.768314,-122.421642,Shizen Vegan Sushi Bar & Izakaya
7,395 Golden Gate Ave,Vegan,37.781473,-122.416836,Golden Era Vegan Restaurant
8,,Vegan,37.791060,-122.401750,WeTheTrillions
9,484 Ellis St,Ethiopian,37.784830,-122.414227,Tadu Ethiopian Kitchen


In [328]:
combined_v_restaurant_coords = tuple(zip(concat_lists['Latitude'], concat_lists['Longitude']))

HeatMap(combined_v_restaurant_coords).add_to(map_sf)
map_sf

In [329]:
two_h_lists = [y_h_combined_df, h_processed_df]
concat_h_lists = pd.concat(two_h_lists, axis = 0, sort = True)
concat_h_lists.drop_duplicates(subset = 'Address', keep = 'first', inplace = True)
concat_h_lists.reset_index(drop=True, inplace = True)

In [330]:
combined_h_restaurant_coords = tuple(zip(concat_h_lists['Latitude'], concat_h_lists['Longitude']))

HeatMap(combined_h_restaurant_coords).add_to(h_map_sf)
h_map_sf

# Result

Vegetarian Results - All of the major restaurant hubs around the city are well equipped to serve Vegetarians/Vegans. In particular Market Street, the Financial District, and Mission Street are the biggest hot spots for such restaurants. It is only in the southern area around Excelsior/Visitation Valley/ Bayshore Heights where vegetarian restaurant coverage becomes rare.

Halal Results - The biggest cluster of Halal restaurants can be found in the area around Powell and Jones St crossing. The second biggest cluster can be found in the Financial District. Elsewhere around the city, Halal restaurants are spread out without much concentration except for Outer Sunset and the Excelsior/Visitation Valley/Bayshore Heights areas where there is nearly no Halal restaurants found.

# Discussion

It is interesting looking at the locations of where the restaurants are located and in the case of both Vegetarian/Vegan and Halal restaurants, both follow the same pattern of having intense hotspots located in the downtown area and then becoming more spreadout the further you move away from there. This would make sense as the downtown area is the area with the most traffic on any given day. Workers who are looking for lunch or people who are looking to get dinner 

One thing to keep in mind is that for a person to have a preference for a Restaurant to commit to the specialty instead of being a restaurant that dabbles in multiple things as evidenced by the FourSquare vs Yelp data. There is some thought to be had about whether a vegetarian would avoid restaurants that have both vegetarian and meat options or they would go to restaurants that are purely Vegetarian only.

One more source of data that I could not find a source for was a list of all the Food Trucks that operate in San Francisco (or any other parts of the bay area for that matter). Food Trucks have become increasingly more common but since it is a mobile operation there is no reliable way to track them down the same way you can track down Restaurants that have a set location. 

The accuracy of FourSquare and Yelp data is pretty good but because they are both crowd sourced data there are many inconsistent results for any given restaurant data. I feel that this is not the fault of FourSquare or Yelp in itself but it is the effect of crowd sourcing your data as you have many different people who report a restaurant possibly inaccurately, without enough information, or just differently from other users

# Conclusion

The combination of Foursquare and Yelp data provided a much better and more accurate picture of all the restaurants that support the category that a person is looking for. In my research I have found that San Francisco is very friendly towards Vegetarians and Vegans all over the city. While there are far less Halal restaurants in San Francisco, there is still plenty of restaurants around the city that can take care of any visitor to the city exploring the more touristy neighbourhoods. 